In [2]:
!pip install nltk

  Cache entry deserialization failed, entry ignored
    100% |████████████████████████████████| 1.5MB 432kB/s ta 0:00:011
  Running setup.py bdist_wheel for nltk ... done
  Stored in directory: /home/chinmay/.cache/pip/wheels/96/86/f6/68ab24c23f207c0077381a5e3904b2815136b879538a24b483
Successfully built nltk


In [1]:
#installing all the libraries needed for the task
import pandas as pd
import json
import nltk
from pandas.io.json import json_normalize
nltk.download('stopwords')
nltk.download('punkt')
import re  
from nltk.corpus import stopwords
stops1 = set(stopwords.words("english"))


[nltk_data] Downloading package stopwords to
[nltk_data]     /home/chinmay/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/chinmay/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
! pip install gensim
! pip install seaborn
! pip install elasticsearch
! pip install esengine
! pip install ipdb

In [2]:
# %cd /home/chinmay/Desktop/TUM/Sem-4/GuidedResearch/MyAttempts/github-code-guided-research/
from Vocabulary import Vocab

In [3]:
%cd /home/chinmay/Desktop/TUM/Sem-4/GuidedResearch/MyAttempts/

/home/chinmay/Desktop/TUM/Sem-4/GuidedResearch/MyAttempts


In [4]:
from preprocess import *
from sub_find import *
from true_subs import *
from constants import *
# from graph_evaluation import *
# from Vocabulary import Vocab
import pickle

In [5]:
from recipe import Recipe
def load_data_from_file(filename=None, ing_file_name=None, local=True, save=False):
    return load_data(file_name=filename, ing_file_name=ing_file_name, local=local, save=save)
    

In [6]:
import os
os.chdir('raw_data')
recipes = load_data_from_file("100k/100ksample", ing_file_name = "dbpedia_ingredients.txt",local=True, save=False )
clean_recipes = [x.proccessed_ing_list for x in recipes]
recipes = load_data_from_file("100k/100ksample2", "dbpedia_ingredients.txt")
sample100k2 = [x.proccessed_ing_list for x in recipes]
clean_recipes.extend(sample100k2) 
type(clean_recipes)

list

In [7]:
# currently looking only on the second set of elements
total_number_of_recipes = len(recipes)
print(total_number_of_recipes)

100000


In [8]:
titles_present = set()
for reduced_recipe_object in recipes:
  titles_present.add(str(reduced_recipe_object.id))

In [9]:
vars(recipes[0])

{'id': 'Cajun Spice Blend',
 'title': 'Cajun Spice Blend',
 'instructions_text': 'mix together and store in airtight container and use as wanted.',
 'ingredients_text': '1 tablespoon garlic powder 1 tablespoon onion powder 1 tablespoon sugar 1 teaspoon salt 1 teaspoon pepper 1 teaspoon dried basil 1 teaspoon paprika 12 teaspoon cayenne pepper',
 'proccessed_ing_list': ['garlic powder',
  'onion powder',
  'sugar',
  'cayenne pepper']}

In [10]:
final_dataset = [(x.proccessed_ing_list, x.id) for x in recipes]
final_dataset[0]

(['garlic powder', 'onion powder', 'sugar', 'cayenne pepper'],
 'Cajun Spice Blend')

In [11]:
# let us use nltk to perform stemming. That way, it is possible to get the matches better
# eg muffins and muffin

In [12]:
from nltk.stem import PorterStemmer
stemmer = PorterStemmer()
# type(titles_present)

In [13]:
def stem_title(word):
    return stemmer.stem(word)


In [14]:
# an example
stem_title('Carsons Chicken Wellington W/ Exotic Mushroom Sherry Sauce')

'carsons chicken wellington w/ exotic mushroom sherry sauc'

In [15]:
updated_title = [stem_title(title) for title in titles_present]

In [16]:
with open('recipe_titles_stemmed.txt', 'w') as file:
    for item in updated_title:
        file.write("%s\n" % item)

In [17]:
my_list = 'cake,pasta,custard,rolls,stew,casserole,sauce,soup,burgers,pizza,salad,muffins,steaks,fish,ham'.split(",")

In [18]:
vindhya_list = ['Cake', 'Salad', 'Noodles', 'Hummus', 'Burger', 'Steak', 'Pie', 'Custard', 'Chips', 'Casserole', 'Pasta',
                'Pizza', 'Omelette', 'Soup', 'Sauce','Sandwich','Muffin','Pickles','Cocktail','Pudding','Wraps','Taco','Daal',
                'Guacamole','Doughnut','Bread','Smoothie','Sorbet','Chili','Stew','Chicken','Beef',]

In [19]:
# check if the recipe matches
# https://stackoverflow.com/questions/6531482/how-to-check-if-a-string-contains-an-element-from-a-list-in-python
counter = 0
for reduced_recipe_object in recipes:
    # Honestly this is a shitty attempt since I already did the stemming earlier
    # however, I am doing the same again
    counter = counter +  any (stem_title(title) in stem_title(reduced_recipe_object.id) for title in vindhya_list)
    
counter

41942

In [20]:
percentage_covered = counter / total_number_of_recipes * 100
print(percentage_covered)

41.942


# Eventually we will be using the cleaned recipe list for our evaluation

In [21]:
# so sample 100k2
final_dataset = [(x.proccessed_ing_list, x.id) for x in recipes]
only_ingredients = [i[0] for i in final_dataset]

In [22]:
type(final_dataset[0])

tuple

## DONE : Fix the code here. I just used glove pretrined so things can be ignored for a while here

In [23]:
ingredient_vocabulary = Vocab()
for recipe in only_ingredients:
    for ingredients in recipe:
        ingredient_vocabulary.add_word(ingredients)

In [24]:
# Let us see the vocabulary size once
ingredient_vocabulary.max_idx 

1152

In [25]:
ingredient_vocabulary.get_word(100)

'hamburger'

In [26]:
import pandas as pd
df = pd.DataFrame(final_dataset, columns =['Ingredients', 'Recipe_id'])

In [27]:
df.iloc[5]

Ingredients    [orange, clove, apple, water, cinnamon]
Recipe_id                     The Spiced Cider Project
Name: 5, dtype: object

## Word2Vec was huge hence glove 100d  file is used ~200MB

#### Use this block only for the first time since it simply does the conversion of the Embeddings. Also, in future, it makes more sense to upload this converted file into colab while running the code

In [28]:
# ! wget -c "https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz"
# ! gzip -d GoogleNews-vectors-negative300.bin.gz
from gensim.test.utils import datapath
from gensim.scripts.glove2word2vec import glove2word2vec
glove_file = datapath('/home/chinmay/Desktop/TUM/Sem-4/GuidedResearch/dataset/glove/glove.6B.100d.txt')
with open('/home/chinmay/Desktop/TUM/Sem-4/GuidedResearch/dataset/glove_w2vformat/glove.100d.w2v', mode='wb+') as dest_file:
    glove2word2vec(glove_file, dest_file)


In [29]:
import gensim
pretrained_embedding = gensim.models.KeyedVectors.load_word2vec_format('~/Desktop/TUM/Sem-4/GuidedResearch/dataset/glove_w2vformat/glove.100d.w2v')

In [30]:
# Just a sample check to indicate things are working as expected
pretrained_embedding.vocab['orange'].index

3200

In [31]:
# This is more of a hack
def recipe_vocab_on_list(ingedient_list):
    result_list = []
    for ingedient in ingedient_list:
        # If multi word ingredient, chose the first word
        ingedient = ingedient.split(" ")[0]
        try:
            result_list.append(pretrained_embedding.vocab[ingedient].index)
        except KeyError:
            result_list.append(pretrained_embedding.vocab['unk'].index)
    return result_list
    

In [32]:
df['Ingredient_Numeric'] = df.Ingredients.apply(recipe_vocab_on_list)

In [33]:
df.iloc[5]

Ingredients           [orange, clove, apple, water, cinnamon]
Recipe_id                            The Spiced Cider Project
Ingredient_Numeric            [3200, 33170, 3292, 430, 17773]
Name: 5, dtype: object

In [34]:
recipe_vocabulary = Vocab()
for recipe in vindhya_list:
    recipe_vocabulary.add_word(stemmer.stem(recipe))

In [35]:
recipe_vocabulary

{'<UNK>': 0, 'cake': 1, 'salad': 2, 'noodl': 3, 'hummu': 4, 'burger': 5, 'steak': 6, 'pie': 7, 'custard': 8, 'chip': 9, 'casserol': 10, 'pasta': 11, 'pizza': 12, 'omelett': 13, 'soup': 14, 'sauc': 15, 'sandwich': 16, 'muffin': 17, 'pickl': 18, 'cocktail': 19, 'pud': 20, 'wrap': 21, 'taco': 22, 'daal': 23, 'guacamol': 24, 'doughnut': 25, 'bread': 26, 'smoothi': 27, 'sorbet': 28, 'chili': 29, 'stew': 30, 'chicken': 31, 'beef': 32}

In [36]:
len(recipe_vocabulary.word_2_idx)

33

In [37]:
recipe_vocabulary.check_any_word_in_vocab(stemmer.stem('The Spiced Cake Project'))
# recipe_vocabulary.get_idx('cake')

(True, 'cake')

In [45]:
# Uptil this point we were simply returning the first match in the recipe vocabulary. 
# This can be modified and we look for all the matches of possible recipe
# Since the list is sorted specific to our rule, we would go ahead and take the minimum value from all
def get_recipe_idx(recipe):
    recipe_stemmed = stemmer.stem(recipe)
    lookup_result = recipe_vocabulary.get_all_words_in_vocab(recipe_stemmed)
    if lookup_result[0]:
        print(lookup_result[1])
        return min(lookup_result[1])
    # No match so 0 returned
    return  0

In [46]:
get_recipe_idx('The Spiced Cake Project')

[1] The Spiced Cake Project


1

In [47]:
df['Recipe_id_numeric'] = df.Recipe_id.apply(get_recipe_idx)

[19] Screwdriver Cocktail
[32, 30] Tomato Beef Stew
[31, 2] Exotic Chicken Salad
[31] Chicken and Vegetable Stir Fry
[2] Citrus-Marinated Shrimp with Grilled-Onion and Orange Salad
[31, 10] Mexican Chicken Casserole
[10] Yumasetta Casserole
[26] Bread and Milk
[31] Country Captain Chicken
[31] Balsamic Roast Chicken Recipe
[12] Chicago Style Pizza (thin crust)
[2] White Bean and Vegetable Salad
[14] Crockpot French Onion Soup
[31] Chef Thomas's Chicken Linguine
[16] Grilled Bacon and Prosciutto Sandwich
[15] Striped Bass in Agrodolce Sauce
[9] Healthy Chocolate Chip Cookies
[11] Food Works How to Cook Pasta
[15] Lemon-Dill Salmon Croquettes with Horseradish Sauce
[31] Chicken Teriyaki
[31, 6] Chicken Fried Steak
[31] Bombay Chicken and Rice
[31, 14] Lots of Veggies Crock Pot Chicken Soup
[15] Pesto Cream Sauce
[31] Cheesy Buffalo Chicken Pockets
[2] Honey Mustard-Turkey Salad
[19] Coffee Cocktail Recipe
[31, 10] Mexican Chicken Casserole
[22, 14] Taco Soup
[31] Chicken Laredo
[26] Chee

[31, 15] Chicken With Sweet and Sour Fig Sauce
[31] Kotopoulo Skorthato (Lemon Garlic Chicken With Potatoes)
[11] Mushroom & Artichoke Pasta
[10] Autumn Carrot Casserole
[2] Greens and Tomato-Tomato Salad
[31, 21] Chicken With Zucchini and Roasted Pepper Wraps
[31] Teriyaki Chicken
[11] Baked Sausage and Mushroom Penne Pasta
[15] Easy Enchilada Sauce
[14] Chunky Cream of Potato Soup
[32] Beef Porcupines Recipe
[14] Golden Peach Soup with Shrimp and Crab Seviche
[6] Pu Pu Platter Steak
[2] Grilled Vegetable Salad
[11] Cheeseburger Pasta
[22, 29] Confetti Taco Chili for 16
[14] Erin's Wild Rice Soup
[6] Tuscan T- Bone Steak or Bistecca Alla Fiorentina
[2] Greek Salad with Tuna
[30] LG SPICY AUBERGINE STEW WITH CORIANDER AND MINT
[9] Ultimate Chocolate Chip Cookies
[31] Chicken Sofrito (Stir-Fried)
[26, 18] Watermelon Rind Bread and Butter Pickles
[31] Down South Chicken Fingers
[31] Chicken with Prunes: Pollo alle Prugne
[31] Pressure-Cooker Chicken Stock
[9] Gluten Free Chocolate Chip P

[16] Braunschweiger Sandwich
[2] Corn Salad
[2] Julie's Pretty Antipasto Salad
[10] Asparagus-Egg Casserole
[31, 14] Chicken Soup
[20] Apple Dumpling Pudding
[7] EZ Peanut Butter Pie II
[2] Herbed Potato Salad
[31] Southern-Style Honey Barbecued Chicken
[26] Corn Bread
[31] Overnight Chicken
[31, 22] Spicy Chicken Tacos
[2] Heirloom Tomato Salad With Shiso and Shallots
[11] Easy White Sauce for Pasta
[9] Chip Dip Recipe
[14] Quick and Easy Loaded Baked Potato Soup #SP5
[15] Chile Dipping Sauce
[26] Sugar-Free Zucchini Bread
[32] Medallions of Beef With Port Sauce and Cashel Blue Cheese
[14] Cabbage Soup I
[2] Italian-Marinated Vegetable Salad
[10] Beans 'n Wiener Biscuit Casserole
[32] Garlic Beef Enchiladas
[10] Squash Casserole
[2, 14] Crunchy Salad with Deep-Fried Yakisoba Noodles & Thick Soup
[31] Chicken Breasts with Fontina and Prosciutto
[11] Greek Pasta
[2] Hawaiian Crab Salad
[31, 12] Buffalo Style Chicken Pizza Recipe
[6] Pan Fried Steak With Vegetables
[31, 2] Kelly's Famous

[16] Grilled Cheese and Onion Sandwiches
[32] Beef Tips and Gravy
[6, 15] Orange Glaze with A.1. Steak Sauce
[26] Coconut & Cashew Bread Paste (Vegan)
[15] My Mom's Tartar Sauce
[29] Grilled Shrimp with Chili and Lime Sauce Kebabs Recipe
[31] Secret chicken seasoning
[17] Zucchini and Quinoa Muffins
[11] Spicy Sausage Pasta
[31] Roasted Chicken with Carrots, Turnips, and Zucchini
[31, 11] Spicy Thai Chicken Pasta
[31] Basil-Red Bell Pepper Chicken Tikka
[2] Albacore Tuna Romaine Salad
[27] Holly Goodness Smoothie
[26] Tarragon Bread Dip
[2] Endive, Mache, and Radish Salad with Champagne Vinaigrette
[15] Horseradish-Apple Sauce
[7] Diet Pineapple Cream Pie Recipe
[2] Green Banana Salad
[2] Gabby Gourmet's Grilled Portabella Salad
[2] Mixed Greens Salad with Pears, Goat Cheese, and Fig Vinaigrette
[16] Shrimp and Italian Style Vegetable Focaccia Sandwich
[32] Green Bean And Beef Casserole Recipe
[32, 29] Vegan Beef Chili
[16] Pesto and Balsamic Waffle Sandwiches
[15] Blackberry-Raspberry

[11] Hamaguri Clam Pasta
[2] Farro Salad
[2] Three Leaf Salad Recipe
[31] 7 Step Chicken Curry
[15] Sublime Hot Fudge Sauce
[2] Crunchy Broccoli Salad
[12] Easiest Pizza Ever
[31] Neely's Jerk Chicken
[31] Creamed Spinach With Garlic Chicken (Crock Pot)
[2] Smoked Salmon Potato Salad With Dill
[17] English Cheddar Muffins
[14] Cheesy Broccoli Soup
[26] Oatmeal Bread with Fig, Anise, and Walnuts
[2] Green Bean, Tomato and Salami Salad
[9] Lemon White Chocolate Chip Blondies
[31, 15] Savory Grilled Chicken Sauce
[31] Caramelized Baked Chicken
[9] Apple-Cream Cheese Chocolate Chip Cookie Bake
[2] Layered Lettuce Salad
[26, 18] Microwave Bread & Butter Pickles
[31] Sriracha Chicken Quesadillas
[3] My Husband's Favorite Soy Sauce Ramen Noodles
[9] Easy Peasy Chocolate Chip Coconut Macaroons
[15] homemade bar-b-que sauce
[1] Grilled Pound Cake with Mexican Chocolate Sauce and Tropical Fruit
[2] Crab Salad
[31] Buffalo Chicken Pita Panini
[12] Fresh Basil-Chicken Pizza
[1] Moist Chocolate Cak

...

[26] Peach Bread
[11] Garden Pasta Bowl
[15] Baked Tilapia With Dill Sauce
[14] Sausage and Lentil Soup
[31, 15] Eggplant, Bell peppers and Chicken with Sweet and Sour Sauce
[26] Sweet Wheat Bread (Bread Machine)
[32, 15] Beef with Gorgonzola Sauce
[32] Beef Brisket
[31] BBQ Chicken Cornbread Pie
[2] Low Fat Roasted Sweet Potato Salad
[14] Bobby Flay's Black Bean Soup
[12] Sun-Dried Tomato, Thyme And Basil Pizza
[31] Chicken Goulash (Chicken Stew)
[7] Vegetable Garden Meat Pie Recipe
[12, 11] Slow Cooker Pizza Pasta
[11, 2] Best Ever!!!!! Italian Pasta Salad
[1] Gluten-Free Vanilla Cake With Cinnamon-Agave Glaze
[31] Chicken Smothered in Gravy
[9] chocolate chip cookies
[31] Breaded Ranch Chicken
[31] Romertopf Roasted Chicken
[26] Newfoundland White Bread
[6] Philly Cheese Steak
[31] Crock Pot Braised Sage Chicken With Stuffing
[15] Caribbean Meatballs With Tropical Sauce
[14] Roasted Cauliflower and Leek Soup
[2] " You Bring the Macaroni Salad," Macaroni Salad
[31] Chicken and Seafoo

[32] Shirley's Braised Beef
[2] Raspberry Spinach Salad
[30] Pinto Bean, Andouille Sausage Stew
[2] Citrus Salad with Arugula and Mint
[2] Southern Salad Rolls
[31, 14] Italian Chicken Vegetable Soup
[6] Mexican Salsa Steak
[2] Lentil Sprout and Bulgur Salad
[17] Sour Cream Cinnamon Nut Muffins
[31] Spicy Roasted Chicken Legs
[26] Sally Lunn Bread
[32] Braised Beef Short Ribs With Ginger Soy Glaze Recipe
[15] Very Special Spaghetti Sauce
[2] Apple-Orange Pinwheel Salad for 2
[8] Slow Cooker Egg Custard
[14] James Martin's mushroom soup with coriander cress recipe
[11] The Pampered Chef Pronto Pasta Bake
[20] Chocolate Fudge Pudding
[31, 30] Coconut Curried Chicken Stew
[12, 26] Bubble-up Pizza Bread
[7, 1] Pumpkin Pie Cake Frosting
[2] Corn and Avocado Salad
[3] Creole Noodles
[31] Thermonuclear Chicken Wings
[10] Cheesy Hash Brown Casserole
[27] Molasses Spice Smoothie
[31] The Best Sour Cream and Bacon Chicken
[16] Chilly, Egg cheese sandwich
[31] Chicken Do Pyaza
[29] Kittencal's Ch

[14] Best Ever Low Fat Mexican Soup
[7] Apple pie baked in apple
[24] Pan-Seared Swordfish with Zesty Guacamole
[28] Mango Sorbet
[1] Flourless Chocolate Cake with Creamy Mousse
[21] Reuben Lettuce Wrap - South Beach Diet
[16] Greek Style Sub Sandwich
[31, 15] Chicken in Tarragon Sauce
[31] Spicy Chicken Wings
[12] Pizza Bianco
[2] Stir Fry Shrimp Salad
[31, 14] Chicken Floating in Curried Coconut Soup
[26] Monterey Jack Bread
[2] Asian Pesto-Shrimp Noodle Salad
[31] Weight Watchers Baked Chicken
[19] Jellybean Cocktail
[15] Herring in Oatmeal with Gooseberry Sauce
[32] Smoked Beef Brisket
[2] Potato Salad with Mustard Vinaigrette
[7] Smokie Link Pie Recipe
[31] Johnny Carino's Five-Cheese Chicken Fettuccine
[17] Nana Wheat Muffins
[7] Banana Pudding Pie (No Bake)
[16] Grilled Veg and Hummus Sandwich
[26] Gourmet Cinnamon Cracker Bread Recipe
[2] Lemon-Dill Dijon Seafood Salad
[31, 2] Chicken and Broccoli Salad
[10] Eggplant (Aubergine) Casserole
[31] Thai Chicken Curry
[2, 16] Salmon 

[31] Paprika Chicken & Rice Bake
[10] Onion Casserole
[2] Broccoli Delight Salad Recipe
[31] Rigatoni With Chicken Ragout
[15] Garlic Pork Tenderloin with Barbecue Sauce
[2] Best of Oregon Salad
[12] Tofu Kimchi Pizza (Really)
[26] Neapolitan Easter Bread
[29] CrockPot Chili
[6] Cubed Steak and Wild Rice
[16] Smoked Turkey BLT Club Sandwich
[14] Broadmoor Hotel Cream of Broccoli Soup
[1, 25] Tim Horton's Style Chocolate Cake Doughnuts
[21] Cheesy Little Link Snack Wraps
[2] Mexican Chopped Salad with Beets and Walnut Dressing
[31, 2] Mini Chicken Patties with Spring Green Salad
[21] Smokey Dijon Parmesan Wrap
[14] Basic Rice Soup
[26] Honey Granola Bread (Abm)
[17] Chocolate-Orange Marbled Muffins
[11, 31] Farfalle (Bow Tie) Pasta With Chicken & Sun-Dried Tomatoes
[14] Spanish White Bean Soup
[32] Beef Diablo
[31] Chicken and Kellogg's Eggo Waffle Appetizer Bites
[31, 2, 21] MIRACLE WHIP Mango Chicken Salad Wrap
[6, 2] Asian Steak Salad
[7] Basic Savory Pie Crust Recipe
[16] Crab Melt 

[32, 30] Beer Beef Stew II
[15] Turkey With Ancho-Orange Sauce
[2] Roberta's Broccoli Salad
[27] Mint and Mango Smoothie
[32] Original Tennessee Barbecue Beef (Crock Pot)
[2] Three Bean Salad
[7, 26] Apple Pie Bread Recipe
[31] Wilma's Curried Chicken Mayonnaise
[31] Weeknight Chicken Pot Pie
[26] Creative Face Bread
[31] Lime and Thai Spices Chicken
[2] Cornbread Salad
[26] Quinoa Bread
[16, 5] Rye Almond Sandwich Loaf and Burger Buns
[32] Mushroom-Sauced Roast Beef
[15] Grilled Lamb T-Bones with a Red Wine Reduction Sauce
[16] Coconut Cookie Ice Cream Sandwiches
[29] Mussel Tomato And Chilies
[2] Nyte's Low-Fat Salad/Potato Salad Dressing
[31] Seven Flavor Chicken
[2] Russian Salad
[15] Alex's Pseudo Tso's Sauce
[3] Ranch Noodles
[2] Uzbek Carrot Salad
[2] Cold Thai Noodle Salad
[12] Pizza Meatloaf
[12] Grilled Mexican Potato and Chorizo Pizza on Tortillas
[31, 12] Chicken Tortilla Pizza
[31] Chicken Salsa Verde Bake
[2] Lentil Salad With Fresh Favas
[26] Pumpkin Cheese Bread I
[9] P

In [54]:
# Shows that the correct result is obtained
df.iloc[4]

Ingredients           [ground beef, tomato, water, green bean, carro...
Recipe_id                                              Tomato Beef Stew
Ingredient_Numeric                 [817, 12333, 430, 989, 20786, 20132]
Recipe_id_numeric                                                    30
Name: 4, dtype: object

In [55]:
df.Recipe_id_numeric.value_counts()

0     64060
31     6151
2      5914
14     3400
15     3058
26     2145
10     1503
32     1341
11     1303
12     1059
29      993
1       981
16      969
17      954
9       869
6       853
7       811
30      759
20      588
27      399
22      378
19      273
3       224
21      205
8       200
24      119
28      113
18       97
4        94
5        90
25       55
13       40
23        2
Name: Recipe_id_numeric, dtype: int64

In [56]:
len(df)

100000

## I am pickling both the vocabularies which would be useful later

In [57]:
with open('vocab_combined.pkl','wb') as file:
    pickle.dump(recipe_vocabulary, file)
    pickle.dump(ingredient_vocabulary, file)

## Let us drop the empty entries

In [58]:
def check_empty(x):
    return len(x) == 0
    

In [59]:
processed_df = df.drop(df[df.Ingredient_Numeric.apply(check_empty)].index)

In [60]:
assert sum(processed_df.Ingredient_Numeric.apply(check_empty)) == 0

In [61]:
# I am just saving the file here for reference later
pd.to_pickle(processed_df, './processed_df_glove.pkl')

## Let us build the two set of data using sklearn dataloaders 

In [44]:
from sklearn.model_selection import train_test_split
df_train, df_test = train_test_split(processed_df, test_size = 0.2, random_state = 0)

# This is the Neural Network portion

In [45]:
# I am assuming that we are able to get the categories here
# Now this should become a LSTM based model which will try and do binary prediction
import torch
import torch.nn as nn
import torch.utils.data as data
import torch.utils.data.dataloader as dataloader
from torch.nn.utils.rnn import pad_sequence

In [46]:
# Some constants
batch_size = 5

In [47]:
# https://github.com/yunjey/seq2seq-dataloader/blob/master/data_loader.py
def collate_fn(data):
    """Creates mini-batch tensors from the list of tuples (src_seq, trg_seq).
    We should build a custom collate_fn rather than using default collate_fn,
    because merging sequences (including padding) is not supported in default.
    Seqeuences are padded to the maximum length of mini-batch sequences (dynamic padding).
    Args:
        data: list of tuple (src_seq, trg_seq).
            - src_seq: torch tensor of shape (?); variable length.
            - trg_seq: torch tensor of shape (?); variable length.
    Returns:
        src_seqs: torch tensor of shape (batch_size, padded_length).
        src_lengths: list of length (batch_size); valid length for each padded source sequence.
        trg_seqs: torch tensor of shape (batch_size, padded_length).
        trg_lengths: list of length (batch_size); valid length for each padded target sequence.
    """
    def merge(sequences):
        lengths = [len(seq) for seq in sequences]
        padded_seqs = torch.zeros(len(sequences), max(lengths)).long()
        for i, seq in enumerate(sequences):
            end = lengths[i]
            padded_seqs[i, :end] = seq[:end]
        return padded_seqs, lengths

    # sort a list by sequence length (descending order) to use pack_padded_sequence
#     print(data[0]) # list of tuples
    data.sort(key=lambda x: len(x[0]), reverse=True)

    # seperate source and target sequences
    src_seqs, trg_seqs = zip(*data)

    # merge sequences (from tuple of 1D tensor to 2D tensor)
    src_seqs, src_lengths = merge(src_seqs)
    # target sequence for us is a single tensor so we do not need to 
    # merge it
    #trg_seqs, trg_lengths = merge(trg_seqs)
    trg_seqs = torch.as_tensor(trg_seqs)
    return src_seqs, src_lengths, trg_seqs #, trg_lengths


In [48]:
class RecipeData(data.Dataset):
    
    def __init__(self, df):
        super(RecipeData, self).__init__()
        self.df = df
    
    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        X = torch.as_tensor(self.df.Ingredient_Numeric.iloc[idx])
        y = torch.as_tensor(self.df.Recipe_id_numeric.iloc[idx])
        return X,y
    

In [49]:
train_dataset = RecipeData(df_train)
test_dataset = RecipeData(df_test)
train_data_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                              batch_size=batch_size,
                                              shuffle=True,
                                              collate_fn=collate_fn,
                                               drop_last=True) # Done for cases when num_samples not exact multiple
test_data_loader = torch.utils.data.DataLoader(dataset=test_dataset, # of the batch_size
                                              batch_size=batch_size,
                                              shuffle=True,
                                              collate_fn=collate_fn,
                                              drop_last=True)

In [50]:
df.Recipe_id_numeric[4]

5

In [51]:
train_dataset[0]

(tensor([ 65,  33, 140,  36]), tensor(0))

In [52]:
for (X,X_len,y) in train_data_loader:
    print(X)
    print(X_len)
    print(y)
    break

tensor([[ 19,   3, 174,  39,  95,  30,  52,  18, 225,   8,  38],
        [130, 281,  38, 721,  24,  69, 129, 287, 278, 131,   0],
        [486, 331,  23, 187,  80, 401, 185, 186, 341,   0,   0],
        [131, 167, 151, 263,  61, 292,  52,  30,   0,   0,   0],
        [129,   3,  94, 137, 196,   0,   0,   0,   0,   0,   0]])
[11, 10, 9, 8, 5]
tensor([1, 0, 0, 0, 0])


In [53]:
X

tensor([[ 19,   3, 174,  39,  95,  30,  52,  18, 225,   8,  38],
        [130, 281,  38, 721,  24,  69, 129, 287, 278, 131,   0],
        [486, 331,  23, 187,  80, 401, 185, 186, 341,   0,   0],
        [131, 167, 151, 263,  61, 292,  52,  30,   0,   0,   0],
        [129,   3,  94, 137, 196,   0,   0,   0,   0,   0,   0]])

In [54]:
class RecipePredictor(nn.Module):
    
    def __init__(self, vocab_size, hidden_dim, embedding_dim, batch_size, output_dim):
        super(RecipePredictor, self).__init__()
        self.vocab_size = vocab_size
        self.output_dim = output_dim
        self.hidden_dim = hidden_dim
        self.embedding_dim = embedding_dim
        self.embedding = nn.Embedding(self.vocab_size, self.embedding_dim)
        self.lstm = nn.LSTM(input_size=self.embedding_dim, hidden_size=self.hidden_dim)
        self.batch_size = batch_size
        self.predictor = nn.Linear(self.hidden_dim, self.output_dim)
        self.non_linearity = nn.ReLU()
        self.init_hidden() # TODO:  This should happen at the beginning of each epoch
        
    def init_hidden(self):
        self.h_n = torch.randn(1, self.batch_size, self.hidden_dim)
        self.c_n = torch.randn(1, self.batch_size, self.hidden_dim)
        
    
    def forward(self, input_sequence, max_len):
#         print(input_sequence)
#         print(max_len)
        embedded = self.embedding(input_sequence)
        packed = nn.utils.rnn.pack_padded_sequence(embedded, max_len, batch_first=True)
        outputs, (self.c_n, self.h_n) = self.lstm(packed, (self.c_n, self.h_n))
        # Unpack padding
        """
            Honestly, I do not know if at this point, I need the output. I would rather prefer to work with the
            self.h_n cell and so will not `pad_padded_sequence`
        """
        #outputs, _ = nn.utils.rnn.pad_packed_sequence(outputs, batch_first=True)
        # on which output should the prediction be done?
        # self.h_n =  num_layers, batch_size, hidden_dim
        batch_size = self.h_n.shape[1]
        output_predicted = self.predictor(self.h_n.reshape(batch_size, -1))
        return output_predicted
        

In [55]:
vocab_size = ingredient_vocabulary.max_idx + 1

In [56]:
num_recipes = recipe_vocabulary.max_idx+1
model = RecipePredictor(vocab_size=vocab_size, hidden_dim=512, embedding_dim=300, batch_size=batch_size,
                       output_dim=num_recipes)

In [57]:
model(X, X_len)

tensor([[ 0.0401,  0.2132,  0.0743, -0.1556,  0.0510,  0.1881, -0.0421,  0.1119,
          0.0713,  0.1193,  0.0347, -0.0798, -0.1863, -0.0639, -0.1612, -0.0156],
        [ 0.0567,  0.0471,  0.0292,  0.0956, -0.0449, -0.0879, -0.0174, -0.1688,
          0.1125, -0.1681,  0.0670, -0.1165,  0.0689,  0.0322, -0.0800,  0.0156],
        [-0.0313,  0.0320, -0.0326,  0.1923,  0.1740, -0.0895, -0.0408,  0.0142,
          0.0165, -0.0118,  0.0636, -0.0233, -0.2884,  0.2130,  0.0235,  0.1105],
        [ 0.1292, -0.0452,  0.1735,  0.2075, -0.1769,  0.0785,  0.1529, -0.1209,
          0.1062,  0.1867,  0.0799, -0.0008,  0.0811, -0.1242,  0.0404, -0.1528],
        [ 0.0271, -0.1232,  0.3260,  0.1704,  0.0706,  0.0453,  0.0249,  0.1153,
          0.2356, -0.0587,  0.2022, -0.1504, -0.0504, -0.0279, -0.0954, -0.0797]],
       grad_fn=<AddmmBackward>)

# Put loss function, gradient and try to train the lstm

In [58]:
lr = 1e-3
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

In [59]:
num_epoch = 2

In [60]:
# Moving things to GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cpu


In [61]:
model.to(device)

RecipePredictor(
  (embedding): Embedding(1153, 300)
  (lstm): LSTM(300, 512)
  (predictor): Linear(in_features=512, out_features=16, bias=True)
  (non_linearity): ReLU()
)

In [62]:
def put_elements_to_device(a,b,c,device):
    return a.to(device), b.to(device), c.to(device)

In [1]:
# !pip install -q tb-nightly

# Load the TensorBoard notebook extension
%load_ext tensorboard

In [74]:
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter()

In [75]:
print(len(train_data_loader) * batch_size)
print(len(df_train))
# Not same since we are dropping some terms which do not match up
N_train = len(train_data_loader) * batch_size
N_test = len(test_data_loader) * batch_size

79945
79948


In [76]:
for epoch in range(num_epoch):
    running_loss = 0
    model.init_hidden()
    for idx,(X,X_len,y) in enumerate(train_data_loader):
        model.zero_grad()
        X, X_len, y = put_elements_to_device(a=X, b=torch.tensor(X_len), c=y, device=device)
        prediction = model(X, X_len)
        loss = criterion(prediction, y)
        loss.backward(retain_graph=True)
        optimizer.step()
        running_loss += loss.item()
        if idx % 100 == 0:
            print("epoch {} loss {}".format(epoch, running_loss))
    writer.add_scalar('Loss/train', running_loss/N_train, epoch)
    # validation set is still left to create
    # Now to test the validation set
    correct = 0
    total = 0
    test_loss = 0
    with torch.no_grad():
        for idx,(X,X_len,y) in enumerate(test_data_loader):
            X, X_len, y = put_elements_to_device(a=X, b=torch.tensor(X_len), c=y, device=device)
            outputs = model(X,X_len)
            loss = criterion(outputs, y)
            test_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += X.size(0)
            correct += (predicted == y).sum().item()
    writer.add_scalar('Loss/train', test_loss/N_test, epoch)
    print('Accuracy of the network on the test samples: %d %%' % (
        100 * correct / total))
        
            

epoch 0 loss 2.0085887908935547
epoch 0 loss 102.51984167844057
epoch 0 loss 221.88543963432312
epoch 0 loss 312.94191985577345
epoch 0 loss 409.68190402537584


KeyboardInterrupt: 

In [67]:
%load_ext tensorboard
%tensorboard --logdir=runs

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [78]:
torch.__version__

'1.3.0'

In [80]:
vindhya_data_df = pd.read_csv('./recipe_title_foodCategory.csv')

In [83]:
list(set(vindhya_data_df.Food_Category))

['Cake',
 'Salad',
 'Noodles',
 'Hummus',
 'Burger',
 'Steak',
 'Pie',
 'Chicken',
 'Custard',
 'Chips',
 'Casserole',
 'Pasta',
 'Pizza',
 'Omelette']

In [94]:
a = [1,2,3,4,5]
b = ['a','b','c','d','e']
c = ['dil','me','dard','e','disco']
sallu_bhoi = zip(a,b,c)

In [95]:
a,b,c = zip(*sallu_bhoi)

In [100]:
np.asarray(a)

array([1, 2, 3, 4, 5])